In [2]:
! pip install dash
! pip install plotly


In [3]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

--2021-11-13 18:18:00--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3033 (3.0K) [text/csv]
Saving to: ‘spacex_launch_dash.csv’

spacex_launch_dash. 100%[===================>]   2.96K  --.-KB/s    in 0s      

2021-11-13 18:18:01 (151 MB/s) - ‘spacex_launch_dash.csv’ saved [3033/3033]



In [4]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [5]:
# Read the airline data into pandas dataframe

spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

marks = {0:'0',1000:'1000kg',2000:'2000kg',3000:'3000kg',4000:'4000kg',5000:'5000kg',6000:'6000kg',7000:'7000kg',8000:'8000kg',9000:'9000kg',10000:'10000kg'}

In [ ]:
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown', 
                                    options=[{'label':'All Sites', 'value':'ALL'},
                                            {'label':'Cape Canaveral Space Launch Complex 40', 'value': 'CCAFS LC-40'},
                                            {'label':'Cape Canaveral Space Launch Complex 40 New', 'value': 'CCAFS SLC-40'},
                                            {'label':'Kennedy Space Center Launch Complex 39', 'value': 'KSC LC-39A'},
                                            {'label':'Vandenberg Space Launch Complex 4','value':'VAFB SLC-4E'}],
                                    placeholder="Select a Launch Site here", 
                                    value = 'ALL',
                                    clearable = False,
                                    searchable=True),
                                html.Br(),

                                # 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, 
                                                step=1000,
                                                value=(min_payload, max_payload),
                                                marks=marks),

                                # 4: Add a scatter chart to show the correlation between payload and launch success
                                #dcc.Graph(id='success-payload-scatter-chart', figure={}),
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# 2: Add a callback function for `site-dropdown` as input, `success-pie-chart` as output

@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value')]
)
def update_graph(option_std):
    print(option_std)
    print(type(option_std))

    df = spacex_df.copy()
    
    if option_std == 'ALL':
        df0 = df[df['class']==1]
        fig = px.pie(df0, values='class', names='Launch Site')
    else:
        df1 = df[df['Launch Site']== option_std]
        fig = px.pie(df1, names='class')

    # fig.show()
    return fig

# 4: Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output

@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
    Input(component_id="payload-slider", component_property="value")]
)
def update_graph(option_std, sliderx):

    df2 = spacex_df.copy()
    df2 = df2[(df2['Payload Mass (kg)'] > sliderx[0]) & (df2['Payload Mass (kg)'] < sliderx[1])]
    if option_std == 'ALL':
        fig = px.scatter(df2, x='Payload Mass (kg)', y='class', color="Booster Version Category")
    else:
        df3 = df2[df2['Launch Site']== option_std]
        fig = px.scatter(df3, x='Payload Mass (kg)', y='class', color="Booster Version Category")
 
    # fig.show()
    return fig


# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
